In [1]:
from konlpy.tag import Komoran
from konlpy.tag import Kkma
import pandas as pd
import numpy as np
from re import match

In [2]:
# data = pd.read_csv("./crawling_result_daum.csv")
data = pd.read_csv("./crawling_result_naver.csv", 
                 names=['제목','작가','평점','장르','연령','한줄 스토리'])
story = data[['한줄 스토리']]

In [3]:
komoran = Komoran()
kkma = Kkma()

keywords = []
temp = []
empty_list = [' ']

# kkma와 komoran 두클래스에서 모두 일반명사이고, 길이가 2이상인 단어들
for i in story.index:
    try:
        val = story._get_value(i,"한줄 스토리")
        words = komoran.pos(val)
        for(text, tclass) in words:
            if tclass=='NNG' and not len(text)==1 and not(match('^[0-9]',text)):
                temp.append(text)
        words = kkma.pos(val)
        for(text, tclass) in words:
            if tclass=='NNG' and not len(text)==1 and not(match('^[0-9]',text)):
                temp.append(text)
        keywords.append(temp.copy())
        temp.clear()
        if (i%10)==0:
            print(i,' ', end='')
    except:
        keywords.append(empty_list.copy())

#중복제거
newkeywords = []
for i in range(len(keywords)):
    newkeywords.append(list(set(keywords[i])))

10  20  30  40  50  60  70  80  90  100  110  120  130  140  150  160  170  180  190  200  210  220  230  240  250  260  270  280  290  300  310  320  330  340  350  360  370  380  390  400  410  420  430  440  450  460  470  480  490  500  510  520  530  540  550  560  570  580  590  600  610  620  630  640  650  660  670  680  690  700  710  720  730  740  750  760  770  780  790  800  810  820  830  840  850  860  870  880  890  900  910  920  930  940  950  960  970  980  990  1000  1010  1020  1030  1040  1050  1060  1070  1080  1090  1100  1110  1120  1130  1140  1150  1160  1170  1180  1190  

In [4]:
# 삭제할 특정 단어 리스트
delete_words = ['이의', '작품', '만화','남자','여자', '자신', '스스로', 
                '의의' ,'사이','남성','여성','만이','작가', '인간', '사람', 
                '시작', '신작', '이번','과의','순간','이야기','']

for i in range(len(newkeywords)):
    for v in newkeywords[i]:
        if v in delete_words:
            newkeywords[i].remove(v)

for i in range(len(newkeywords)):
    for v in newkeywords[i]:
        if v in delete_words:
            newkeywords[i].remove(v)

In [31]:
# 학원물
school_words = ['학교','학원','전교','교실','교내','중학생','고등학생','여고생','여학생','남학생','전학생','전학','고교']
isSchool = []
next_webtoon = False
for i in range(len(newkeywords)):
    for v in newkeywords[i]:
        if v in school_words:
            isSchool.append(1)
            next_webtoon = True
            break
    if next_webtoon:
        next_webtoon = False
        continue
    else:
        isSchool.append(0)

In [6]:
for i in range(len(newkeywords)):
    print(i,': ', end='')
    print(newkeywords[i])

0 : ['한번', '인생', '내가', '학교']
1 : ['혼자', '처음']
2 : ['생각', '머릿속', '세포', '유미']
3 : ['취향', '내성', '소신', '비밀', '성향', '내성적']
4 : ['세상', '체육관', '진정', '학원', '무도', '그릇', '액션물', '실천', '한림', '영하']
5 : ['불명', '아이', '정체불명', '부부', '일상', '신묘']
6 : ['활약', '영웅', '난세', '조조', '메신저', '방송', '소통', '삼국지', '군사', '개인', '모집', '핸드폰']
7 : ['이름', '선천적', '백사', '소년', '선천', '두뇌', '도구', '이용', '심리']
8 : ['얼굴', '메이크업', '아티스트', '천재', '대학생', '뮤즈']
9 : ['다정', '능력', '운명', '지옥', '언니', '하루', '소녀', '천재', '은정']
10 : ['복수', '사랑', '세상', '최강자', '치사율', '대표작', '희귀', '두려움', '소녀', '가슴', '최강']
11 : ['논문', '엄마', '기반', '육아', '임신', '최고', '박사', '대로', '아빠', '좌충우돌', '출산', '발모제', '의사', '난방', '항문', '산부인과', '모제', '우여곡절', '공학', '완벽', '앞길', '중구난방']
12 : ['이상', '오늘', '동안', '이상형', '노력']
13 : ['마음', '엘리트', '집단', '가야', '시대', '삼국', '전학생']
14 : ['기안', '세상', '청춘', '가의']
15 : ['군대', '충성']
16 : ['처음', '패배', '여주', '내기', '승자', '한번', '기고']
17 : ['마음', '감성', '리얼', '동갑', '커플', '서울', '부산', '메리', '터프', '외모', '소녀', '가이', '일기', '청순', '연애']
18 : ['세계', '제임스', '막장', 

1059 : ['사기', '떼돈', '점철', '점쟁이', '냉소', '주의자', '배신', '냉소주의', '사기꾼']
1060 : ['변호사', '동네', '본격', '생활', '법정']
1061 : ['프란시스', '태권도', '각양각색', '태권도장']
1062 : ['성장', '아이']
1063 : ['루시퍼', '쟁탈전', '경호원', '구슬']
1064 : ['컨텐츠', '경력', '감독', '당선작', '스카우트', '하이', '고교', '드림', '공백기', '축구', '축구공', '백수', '서울', '축구팀', '한번']
1065 : ['세계', '인상', '대학', '최강자', '취미', '진출작', '진출', '지배자']
1066 : ['미지', '오묘', '신비', '동화']
1067 : ['드라마', '가족', '실종', '미스터리', '예측', '마지막', '불가']
1068 : ['대학', '최강자', '그림자', '어로', '안지', '진출작', '동료', '진출']
1069 : ['당선작', '슈퍼', '이별']
1070 : ['마을', '전염', '최초', '화상', '성장기', '상황', '소개', '사춘기', '어르신', '소녀', '최고', '생애', '화재', '좀비', '힐링', '나의', '흉터']
1071 : ['내일']
1072 : ['사랑', '운명', '우연', '가슴']
1073 : ['벼슬', '벼슬자리', '조선', '자리', '관리', '만만디', '지망', '지망생', '후기']
1074 : ['기사', '싸움', '스릴']
1075 : ['특정', '세상', '소년', '학원', '주제', '장애', '범죄자', '스릴러', '장애인']
1076 : ['친구', '솔과', '성장', '고등학생', '공포증']
1077 : ['수사', '사고', '후유증', '안대', '조금', '그때', '이후']
1078 : ['양이', '사회', '중립', '입사', '천사', '초년', '초년생', '일상',

In [32]:
data['스토리 키워드'] = newkeywords
data['학원물'] = isSchool

In [33]:
# data.to_csv("./storykeywords_result_daum.csv", header=True, index = False)
data.to_csv("./storykeywords_result_naver.csv", header=True, index = False)